<a href="https://colab.research.google.com/github/aflah02/BackTranslation-Based-Data-Augmentation/blob/main/BacktranslationUsingPivotLanguage.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install translators
import pandas as pd
import translators as ts

     |████████████████████████████████| 6.4 MB 6.8 MB/s 
     |████████████████████████████████| 81 kB 9.1 MB/s 
     |████████████████████████████████| 58 kB 5.3 MB/s 
     |████████████████████████████████| 63 kB 1.5 MB/s 
     |████████████████████████████████| 65 kB 3.4 MB/s 
  Created wheel for PyExecJS: filename=PyExecJS-1.5.1-py3-none-any.whl size=14598 sha256=a1059f51e1c89c3a3cc509347e8627f4e654b942fa7fe9283e14d06f07e86701
  Stored in directory: /root/.cache/pip/wheels/9a/ee/03/da5c0b4a8c13362beeb844eb913bbe58a89bde1de2b9157007
Successfully built PyExecJS
  Attempting uninstall: requests
    Found existing installation: requests 2.23.0
    Uninstalling requests-2.23.0:
      Successfully uninstalled requests-2.23.0
  Attempting uninstall: lxml
    Found existing installation: lxml 4.2.6
    Uninstalling lxml-4.2.6:
      Successfully uninstalled lxml-4.2.6
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour i

Using United States server backend.


In [ ]:
def translator_constructor(api):
    if api == 'google':
        return ts.google
    elif api == 'bing':
        return ts.bing
    elif api == 'baidu':
        return ts.baidu
    elif api == 'sogou':
        return ts.sogou
    elif api == 'youdao':
        return ts.youdao
    elif api == 'tencent':
        return ts.tencent
    elif api == 'alibaba':
        return ts.alibaba
    else:
        raise NotImplementedError(f'{api} translator is not realised!')

### Dataset Download

In [2]:
!wget -q https://raw.githubusercontent.com/clairett/pytorch-sentiment-classification/master/data/SST2/train.tsv
!wget -q https://raw.githubusercontent.com/clairett/pytorch-sentiment-classification/master/data/SST2/test.tsv
!wget -q https://raw.githubusercontent.com/clairett/pytorch-sentiment-classification/master/data/SST2/dev.tsv

### Dataset Building Function

In [3]:
def buildTrainTestDatasets(doAug, NumberOfTrainingSamplesTouse, pivotLanguageCode, API, currentLanguageCode):
  df_train = pd.read_csv('train.tsv', delimiter = '\t', names = ['Sentence', 'Label'])
  df_dev = pd.read_csv('dev.tsv', delimiter = '\t', names = ['Sentence', 'Label'])
  df_test = pd.read_csv('test.tsv', delimiter = '\t', names = ['Sentence', 'Label'])

  df_train = pd.concat([df_train, df_dev])
  df_train = df_train.reset_index(drop=True)
  df_train = df_train.loc[:, ["Label", "Sentence"]]
  df_test = df_test.loc[:, ["Label", "Sentence"]]
  if (NumberOfTrainingSamplesTouse > len(df_train)):
    raise Exception("More Samples Asked For Than Present")

  df_train = df_train.iloc[:NumberOfTrainingSamplesTouse]
  if (doAug):
    convertedSentences = []
    originalSentences = df_train["Sentence"].to_list()
    originalLabels = df_train["Label"].to_list()
    translateFunction = translator_constructor(API)
    for sentence in originalSentences:
      convertedSentences.append(translateFunction(translateFunction(sentence, currentLanguageCode, pivotLanguageCode), pivotLanguageCode, currentLanguageCode))
    newSentenceSet = originalSentences + convertedSentences
    newLabelSet = originalLabels + originalLabels
    df_train = pd.DataFrame(
        {'Label': newSentenceSet,
        'Sentence': newLabelSet
         })
  return df_train, df_test

In [ ]:
df_train, df_test = buildTrainTestDatasets(True, 100, 'es', 'google', 'en')